In [2]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [8]:
import gradio as gr

In [5]:
MODEL = "llama3.2"
ollama_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [6]:
message = [{"role":"user", "content": "Tell me a joke on managers"}]

In [7]:
response = ollama_openai.chat.completions.create(model=MODEL,
                                                 messages=message)
print(response.choices[0].message.content)

Why did the manager bring a ladder to the meeting?

Because he wanted to take his decisions to a higher level!


In [9]:
system_message = "You are a helpful assistant"

In [13]:
def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = ollama_openai.chat.completions.create(model=MODEL, messages=messages)
    return completion.choices[0].message.content

In [15]:
message_gpt("When India won the cricket world cup?")

'India has won the Cricket World Cup twice:\n\n1. 1983 Cricket World Cup: India won its first World Cup under the captaincy of Kapil Dev and Sunil Gavaskar.\n2. 2011 Cricket World Cup was not part of Indian teams although they beat Pakistan one day cricket world cup held at Mumbai and then defeated the west indies to reach the final\n\n3. 2019 Cricket World Cup: India won its second World Cup under the captaincy of Virat Kohli, defeating New Zealand in the final.\n\nSo, the two instances when India won the Cricket World Cup are:\n\n- 1983 (under Kapil Dev)\n- None prior the super-over win 2011 \n   - the 2019 world cup (under Virat kohli)'

In [16]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [17]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input hello bhai how are you?
Shout has been called with input OK


In [21]:
#gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input shout


In [26]:
view = gr.Interface(
    fn=message_gpt,
    inputs=gr.Textbox(label="Your message", lines=6),
    outputs=gr.Textbox(label="Response:", lines=8),
    flagging_mode="never"
)
view.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [ ]:
system_message1 = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = ollama_openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch(inbrowser=True)

In [36]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    #elif model=="Claude":
        #result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch(inbrowser=True)

***Now let's try gradio on the brochure***

In [38]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [39]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [40]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    #elif model=="Claude":
        #result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch(inbrowser=True)